In [ ]:
# import libraries
import keras
from utils.preprocess import *
from config import *

In [ ]:
# load dataset
dataset = load_unipen_dataset(no_cap)

# normalize dataset
def normalize(data, label):
    return tf.cast(data, tf.float32) / 255.0, label

if normalize_dataset:
    dataset = dataset.map(normalize)
dataset = dataset.shuffle(10000).batch(64)

In [ ]:
# load model
target_model = "unipen_no_cap_model" if no_cap else "unipen_model"
model = keras.models.load_model(f"data/{target_model}.h5")

In [ ]:
# predict all
loss, acc = model.evaluate(dataset)
print("Overall loss:    ", loss)
print("Overall accuracy:", acc)

In [ ]:
from keras.models import Model
import numpy as np

# Get the outputs of all layers
layer_outputs = [layer.output for layer in model.layers]

# Create a new model that outputs the outputs of all layers
model_all_outputs = Model(inputs=model.input, outputs=layer_outputs)

# Initialize lists to keep track of the largest and smallest values for each layer
max_value = float("-inf")
min_value = float("inf")

# Predict with the new model
outputs = model_all_outputs.predict(dataset.take(100))

# Loop through each output in outputs
for output in outputs:
    # Loop through each layer
    for layer_index, layer in enumerate(output):
        # Find the largest value in the layer
        layer_max = np.max(layer)
        # Find the smallest value in the layer
        layer_min = np.min(layer)
        # Update the max and min values for this layer if it's greater or less than the current max or min
        max_value = max(layer_max, max_value)
        min_value = min(layer_min, min_value)

# Print the largest and smallest values for each layer
print("Max:", max_value)
print("Min:", min_value)

In [ ]:
# predict & plot
from utils.demonstration import plot_digits

for img, label in dataset.take(1):
    prediction = model.predict(img, verbose=0)
    plot_digits(img.numpy(), label.numpy(), prediction, (8, 8))